In [1]:
from bs4 import BeautifulSoup
from requests import get

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait

import time
import os

import json

In [2]:
BASE_URL = "https://tarikh.inoor.ir/"

SOURCE = 11970
EVENT_TYPES = {"BIRTH": 2, "JUDICIAL": 44, "CULTURAL": 1070}

ROOT = "DATASET-V1"

In [3]:
def get_events_page(
    event_type: int,
    source: int,
    page_number: int,
    base_url: str = BASE_URL,
    timeout: int = 5,
) -> BeautifulSoup:
    EVENT_PAGE_POSTFIX = "fa/event/list/رویدادها"
    params = {
        "UserLanguage": "fa",
        "EventType": str(event_type),
        "Source": str(source),
        "PageNumber": str(page_number),
    }

    EVENTS_PAGE_URL = f"{base_url}{EVENT_PAGE_POSTFIX}"
    query_string = "&".join(f"{k}={v}" for k, v in params.items())
    url = f"{EVENTS_PAGE_URL}?{query_string}"

    options = webdriver.ChromeOptions()
    options.add_argument("--headless")

    driver = webdriver.Chrome(options=options)

    try:
        driver.get(url)
        WebDriverWait(driver, timeout)

        return BeautifulSoup(driver.page_source, "html.parser")
    finally:
        driver.quit()

In [4]:
def get_events_links(events: BeautifulSoup) -> list[str]:
    return [
        a.get("href").split("?")[0]
        for a in events.select("div.event-title a")
        if a and a.get("href")
    ]

In [5]:
def get_event_page(
    link: str, base_url: str = BASE_URL, timeout: int = 5
) -> BeautifulSoup:
    url = f"{base_url}{link[1:]}"

    options = webdriver.ChromeOptions()
    options.add_argument("--headless")

    with webdriver.Chrome(options=options) as driver:
        driver.get(url)
        WebDriverWait(driver, timeout)

        return BeautifulSoup(driver.page_source, "html.parser")

In [6]:
def to_persian_digits(s: str) -> str:
    TRANSLATION_TABLE = str.maketrans("0123456789", "۰۱۲۳۴۵۶۷۸۹")
    return s.translate(TRANSLATION_TABLE)

In [7]:
def sanitize(s: str) -> str:
    unwanted_chars = ["؟", "#", "$", "%", "&", "*", "(", ")"]
    for char in unwanted_chars:
        s = s.replace(char, "")
    return s

In [8]:
def get_location_name(s: str) -> str:
    return s.split("(")[0].strip()

In [9]:
def remove_non_digit_chars(s: str) -> str:
    return "".join(filter(str.isdigit, s))

In [10]:
def process_pages(type: str, pages):
    os.makedirs(f"./{ROOT}", exist_ok=True)
    os.makedirs(f"./{ROOT}/{type}", exist_ok=True)

    locations = json.load(open("./cities.json"))

    global_idx = 1

    processed_links = set()

    for page_number in pages:
        page = get_events_page(
            event_type=EVENT_TYPES[type],
            source=SOURCE,
            page_number=page_number,
        )

        links = get_events_links(page)

        for link in links:
            if link in processed_links:
                continue
            processed_links.add(link)

            try:
                event = get_event_page(link, timeout=2)

                title = event.find("h1", class_="event-title").text.strip()
                comment = event.find("h3", class_="event-comment").text.strip()
                description = event.find("p", class_="report-text").text.strip()
                source = event.find("div", class_="report-source").text.strip()

                row_values = event.select(".row-value")
                try:
                    year = int(remove_non_digit_chars(row_values[0].text))
                    year = year - ((year * 11) // 365)
                except:
                    year = "<TBD>"

                try:
                    city = " ".join(row_values[2].text.replace("\n", "").split())
                    city = get_location_name(to_persian_digits(city))
                except:
                    city = "<TBD>"

                # save data
                location = locations.get(city, -1)

                with open(
                    f"./{ROOT}/{type}/{global_idx:03d}.json", "w", encoding="utf-8"
                ) as f:
                    json.dump(
                        {
                            "title": to_persian_digits(comment),
                            "description": to_persian_digits(description),
                            "period": {
                                "start_year": to_persian_digits(str(year)),
                                "end_year": to_persian_digits(str(year)),
                            },
                            "location": (
                                {
                                    "province": "<TBD>",
                                    "city": to_persian_digits(city),
                                    "coordinates": {
                                        "latitude": "<TBD>",
                                        "longitude": "<TBD>",
                                    },
                                }
                                if location == -1
                                else location
                            ),
                            "refrences": {
                                "title": to_persian_digits(source),
                                "author": "حسن فراهانی",
                                "year": "۱۳۸۵",
                            },
                            "source": {
                                "title": to_persian_digits(title),
                                "author": "پایگاه جامع تاریخ",
                                "url": f"{BASE_URL}{link[1:]}",
                            },
                        },
                        f,
                        ensure_ascii=False,
                        indent=4,
                    )

                print(f"✅ {global_idx:03d} - {link}")
                time.sleep(0.5)

                global_idx += 1
            except:
                print(f"☹️ ERROR PROCESSING LINK: {link}")

In [11]:
process_pages('BIRTH', range(1, 7 + 1))

✅ 001 - /fa/event/page/1KKKKM/ولادت_شیخ_عبد_النبی_نوری
✅ 002 - /fa/event/page/2E0E8I/ولادت_جهانشاه_خان_امیر_افشار
✅ 003 - /fa/event/page/EK839E/ولادت_سید_اسد_الله_خرقانی
✅ 004 - /fa/event/page/FD775F/ولادت_مهدی_قلی_خان_سدید_الملک
✅ 005 - /fa/event/page/1K999L/ولادت_عبد_الصمد_میرزا_قاجار
✅ 006 - /fa/event/page/1L395K/ولادت_مسعود_میرزا_ظل_السلطان
✅ 007 - /fa/event/page/2E3J7J/ولادت_مظفر_الدین_شاه_قاجار
✅ 008 - /fa/event/page/1K361D/ولادت_امیر_مؤید_سوادکوهی
✅ 009 - /fa/event/page/1L02MM/ولادت_فتح_الله_اکبر
✅ 010 - /fa/event/page/1L395J/ولادت_کسرائیل_خانم_بانو_عظمی
✅ 011 - /fa/event/page/1K355M/ولادت_ضیاء_الواعظین
☹️ ERROR PROCESSING LINK: /fa/event/page/2E358J/ولادت_نظم_الدوله
☹️ ERROR PROCESSING LINK: /fa/event/page/FD34DF/ولادت_میرزا_محمد_ارباب_قمی
✅ 012 - /fa/event/page/2D627E/ولادت_آیت_الله_شیخ_عبد_الکریم_حائری
✅ 013 - /fa/event/page/1L044K/ولادت_میرزا_یحیی_امام_جمعه_خویی
✅ 014 - /fa/event/page/1K2M2M/ولادت_اسحاق_خان_مفخم_الدوله
✅ 015 - /fa/event/page/1LL47J/ولادت_اسماعیل_معتصم_الملک


In [11]:
process_pages('JUDICIAL', range(1, 5 + 1))

✅ 001 - /fa/event/page/1K132L/احضار_و_محاکمه_معین_الرسائل
✅ 002 - /fa/event/page/1K132K/احضار_مسئولان_صندوق_مالیه
✅ 003 - /fa/event/page/1K132M/احضار_اعتضاد_همایون
✅ 004 - /fa/event/page/1K239H/دستور_رسیدگی_به_اتباع_لهستانی_زندانی_در_کنسولگری_روسیه
✅ 005 - /fa/event/page/1K23KL/رسیدگی_به_اتهامات_کامیل_مولیتور
✅ 006 - /fa/event/page/EK238E/تبرئه_سید_ابو_الحسن_نائینی
✅ 007 - /fa/event/page/1K238I/تبرئه_معین_الرسائل
✅ 008 - /fa/event/page/1K29KK/تازیانه_زدن_و_تبعید_پزشک_کلیمی_به_جرم_توهین_به_اسلام
✅ 009 - /fa/event/page/1K352H/رسیدگی_به_عملکرد_میرزا_یوسف_خان_قائم_مقام
✅ 010 - /fa/event/page/1K5K4J/رسیدگی_به_عملکرد_میرزا_یوسف_خان_قائم_مقام
✅ 011 - /fa/event/page/1K5M1M/اعدام_سالار_محتشم
✅ 012 - /fa/event/page/1K585K/اعدام_اشرار_زنجان
✅ 013 - /fa/event/page/1KJ0JJ/محاکمه_نیروهای_خالو_قربان
✅ 014 - /fa/event/page/2E3K5K/عفو_عبد_القدیر_آزاد_و_زبردست_خان
✅ 015 - /fa/event/page/1K633H/توقیف_روزنامه_اقتصاد_ایران
✅ 016 - /fa/event/page/1K644G/دستور_اعدام_قاتلان_نظامیان
✅ 017 - /fa/event/page/1KK4

In [14]:
process_pages('CULTURAL', range(1, 10 + 1))

✅ 001 - /fa/event/page/2ED24D/فعالیت_مبلغان_کاتولیک_فرانسوی_در_غرب_آذربایجان
✅ 002 - /fa/event/page/2E42JJ/تأسیس_دبستان_دخترانه_آمریکایی_در_ارومیه
✅ 003 - /fa/event/page/2E426E/تأسیس_مدرسه_شبانه_روزی_آمریکایی_در_آذربایجان
✅ 004 - /fa/event/page/FE0F4F/تأسیس_مدرسه_سن_لویی_در_تهران
✅ 005 - /fa/event/page/2E428G/گسترش_دامنه_فعالیت_میسیون_آمریکایی_در_ایران
✅ 006 - /fa/event/page/2E428J/تأسیس_دبستان_ارامنه_توسط_میسیون_آمریکایی_در_ایران
✅ 007 - /fa/event/page/2D3JDJ/انتشار_روزنامه_الاهرام
✅ 008 - /fa/event/page/2E426M/تأسیس_مدرسه_شبانه_روزی_دخترانه_آمریکایی_در_آذربایجان
✅ 009 - /fa/event/page/1LLJ1J/تحصیل_مخبر_السلطنه_در_آلمان
✅ 010 - /fa/event/page/1K348J/تأسیس_آلیانس_فرانسه_کانون_ملی_برای_ترویج_زبان_فرانسه
✅ 011 - /fa/event/page/2E5E2M/تحصیل_ارباب_کیخسرو_در_تهران
✅ 012 - /fa/event/page/2E375L/انتشار_روزنامه_آزاد
✅ 013 - /fa/event/page/1KK66D/تحصیل_سید_حسین_طباطبائی_قمی_نزد_میرزای_شیرازی
✅ 014 - /fa/event/page/1LL61K/تدریس_زبان_آلمانی_توسط_مخبر_السلطنه_در_دار_الفنون
✅ 015 - /fa/event/page/F